In [10]:
import pandas as pd
import os
from skimage import io
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, precision_score, recall_score, classification_report, f1_score, roc_auc_score
import pickle
import Functions2
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score, classification_report

In [2]:
# Load the data
data_path = r"C:\Users\serru\OneDrive\Documents\Project2\Project-2-Medical-Imaging\data\full_data.csv"
df = pd.read_csv(data_path)

# Preprocess the diagnostic column
df['diagnostic'] = df['diagnostic'].map({'BCC': 1, 'MEL': 1, 'SCC': 1, 'ACK': 0, 'NEV': 0, 'SEK': 0})

In [3]:
# Define the function to extract features
def extract_features(folder_path):
    feature_1 = []
    feature_2 = []
    feature_3 = []
    feature_4 = []
    feature_5 = []
    feature_6 = []
    feature_7 = []

    for filename in os.listdir(folder_path):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            image_path = os.path.join(folder_path, filename)
            original = io.imread(image_path)

            # Ignore the alpha channel (e.g. transparency)
            if original.shape[-1] == 4:
                original = original[..., :3]

            feature_1.append(Functions2.measure_pigment_network(original))
            feature_2.append(Functions2.measure_blue_veil(original))
            feature_3.append(Functions2.measure_vascular(original))
            feature_4.append(Functions2.measure_globules(original))
            feature_5.append(Functions2.measure_streaks(original))
            feature_6.append(Functions2.measure_irregular_pigmentation(original))
            feature_7.append(Functions2.measure_regression(original))

    return feature_1, feature_2, feature_3, feature_4, feature_5, feature_6, feature_7

In [4]:
# Define the folder path for image processing
folder_path_in = r"C:\Users\serru\OneDrive\Documents\Project2\Project-2-Medical-Imaging\data\ColorMask\Training"

# Extract features from the images
feature_1, feature_2, feature_3, feature_4, feature_5, feature_6, feature_7 = extract_features(folder_path_in)

In [5]:
# Create a DataFrame for the features
features_df = pd.DataFrame()
features_df["img_id"] = [filename for filename in os.listdir(folder_path_in) if filename.endswith(('.jpg', '.png'))]
features_df["1: pigment network"] = feature_1
features_df["2: Blue veil"] = feature_2
features_df["3: Vascular"] = feature_3
features_df["4: Globules"] = feature_4
features_df["5: Streaks"] = feature_5
features_df["6: Pigmentation"] = feature_6
features_df["7: Regression"] = feature_7

# Merge the features DataFrame with the diagnostic column from the original DataFrame
df_merged = pd.merge(df[['img_id', 'diagnostic']], features_df, on='img_id', how='inner')

In [6]:
# Split the data into training and testing sets
X = df_merged.drop(['img_id', 'diagnostic'], axis=1)
Y = df_merged['diagnostic']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.5)

In [11]:
# Logistic Regression
logreg_classifier = LogisticRegression()
logreg_classifier.fit(X_train, Y_train)
logreg_prediction = logreg_classifier.predict(X_test)

# Evaluate Logistic Regression
logreg_cm = confusion_matrix(Y_test, logreg_prediction)
logreg_precision = precision_score(Y_test, logreg_prediction)
logreg_recall = recall_score(Y_test, logreg_prediction)
logreg_f1 = f1_score(Y_test, logreg_prediction)
logreg_auc_roc = roc_auc_score(Y_test, logreg_prediction)
logreg_classification_rep = classification_report(Y_test, logreg_prediction)

# Print Logistic Regression evaluation metrics
print("Logistic Regression:")
print("Confusion Matrix:")
print(logreg_cm)
print("Precision:", logreg_precision)
print("Recall:", logreg_recall)
print("F1 Score:", logreg_f1)
print("AUC-ROC Score:", logreg_auc_roc)
print("Classification Report:")
print(logreg_classification_rep)

Logistic Regression:
Confusion Matrix:
[[13  9]
 [ 6 16]]
Precision: 0.64
Recall: 0.7272727272727273
F1 Score: 0.6808510638297872
AUC-ROC Score: 0.659090909090909
Classification Report:
              precision    recall  f1-score   support

           0       0.68      0.59      0.63        22
           1       0.64      0.73      0.68        22

    accuracy                           0.66        44
   macro avg       0.66      0.66      0.66        44
weighted avg       0.66      0.66      0.66        44



In [27]:
# Save LR model
with open(r'C:\Users\serru\OneDrive\Documents\Project2\Project-2-Medical-Imaging\Pickle\logreg_model.pkl', 'wb') as file:
    pickle.dump(logreg_classifier, file)

# Save KNN model
with open(r'C:\Users\serru\OneDrive\Documents\Project2\Project-2-Medical-Imaging\Pickle\knn_model.pkl', 'wb') as file:
    pickle.dump(knn_classifier, file)

In [40]:
# Predict the diagnostic for the test set
test_path = r"C:\Users\serru\OneDrive\Documents\Project2\Project-2-Medical-Imaging\data\ColorMask\Test"
test_feature_1, test_feature_2, test_feature_3, test_feature_4, test_feature_5, test_feature_6, test_feature_7 = extract_features(test_path)
test_features_df = pd.DataFrame()
test_features_df["img_id"] = [filename for filename in os.listdir(test_path) if filename.endswith(('.jpg', '.png'))]
test_features_df["1: pigment network"] = test_feature_1
test_features_df["2: Blue veil"] = test_feature_2
test_features_df["3: Vascular"] = test_feature_3
test_features_df["4: Globules"] = test_feature_4
test_features_df["5: Streaks"] = test_feature_5
test_features_df["6: Pigmentation"] = test_feature_6
test_features_df["7: Regression"] = test_feature_7

# Load the model
with open(r'C:\Users\serru\OneDrive\Documents\Project2\Project-2-Medical-Imaging\Pickle\logreg_model.pkl', 'rb') as file:
    logreg_model = pickle.load(file)

# Predict the diagnostic for the test set
test_X = test_features_df.drop(['img_id'], axis=1)
test_Y = logreg_model.predict(test_X)

# Create a DataFrame for the predictions
test_predictions_df = pd.DataFrame()
test_predictions_df["img_id"] = test_features_df["img_id"]
test_predictions_df["diagnostic"] = test_Y

# Print the predictions
print(test_predictions_df)

                  img_id  diagnostic
0    PAT_109_868_723.png           1
1    PAT_236_361_180.png           0
2      PAT_26_37_865.png           0
3    PAT_270_417_257.png           1
4   PAT_333_1469_499.png           0
5      PAT_38_54_234.png           1
6    PAT_398_799_163.png           0
7       PAT_42_58_13.png           1
8      PAT_56_86_479.png           1
9      PAT_59_46_537.png           0
10   PAT_76_1039_269.png           1
11    PAT_998_17_641.png           0
